In [5]:
import solcore

# Composition parameter for Al0.3Ga0.7As
x = 0.3

# Carrier type (n for electrons, p for holes)
carrier_type = 'n'  # Calculate electron mobility

# Temperature in Kelvin
temperature = 300  # Example temperature

# Calculate the carrier mobility
mobility = solcore.material_data.mobility.calculate_AlGaAs(x, carrier_type, temperature)

# Print the result
print("Carrier mobility for Al0.3Ga0.7As ({}-type) at {} K: {} cm^2/Vs".format(carrier_type, temperature, mobility))

ModuleNotFoundError: No module named 'solcore'

In [4]:
!pip install solcore

'pip' is not recognized as an internal or external command,
operable program or batch file.
